In [3]:
import tensorflow as tf
import numpy as np
import math
import tflearn

In [2]:
!pip install tflearn

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 98 kB 943 kB/s eta 0:00:01
  Created wheel for tflearn: filename=tflearn-0.3.2-py3-none-any.whl size=128208 sha256=af58cadf310034a7a10850ebb527be26a21c3af36ae0da27ff0f30bba638faa3
  Stored in directory: /Users/jangalask/Library/Caches/pip/wheels/8e/b7/55/b387552a28a2fefa559a362ffe17c97fac7e618ef948930b3c
Successfully built tflearn


In [4]:
#   Create data in straight Python, create some size(x) and price(y) data points, using price = (m * size) + b.  
#        Here b is a price base based on other factors

#  generation some house sizes between 1000 and 3500 (typical sq ft of house)
num_house = 160
np.random.seed(42)
house_size = np.random.randint(low=1000, high=3500, size=num_house )

In [5]:
# Generate house prices from house size with a random noise added+.
np.random.seed(42)
house_price = house_size * 100.0 + np.random.randint(low=20000, high=70000, size=num_house)

In [6]:
# you need to normalize values to prevent under/overflows.
def normalize(array):
    return (array - array.mean()) / array.std()


In [7]:
# 1. Get Data
# Split the data into training and testing, and normalized the data 

# define number of training samples, 0.7 = 70%.  We can take the first 70% since the values are randomized
num_train_samples = math.floor(num_house * 0.7)

In [8]:
# define training data
train_house_size = np.asarray(house_size[:num_train_samples])
train_price = np.asanyarray(house_price[:num_train_samples:])

train_house_size_norm = normalize(train_house_size)
train_price_norm = normalize(train_price)

# define test data
test_house_size = np.array(house_size[num_train_samples:])
test_house_price = np.array(house_price[num_train_samples:])

test_house_size_norm = normalize(test_house_size)
test_house_price_norm = normalize(test_house_price)

In [9]:
#one value, one value out. Names let u see in tensorboard
input = tflearn.input_data(shape=[None], name="InputData") # Input data is a list of undefined length
linear = tflearn.layers.core.single_unit(input, activation='linear', name='Linear')
#ddefine a single neuron with linear activation. The euqation, it is linear(Wx +b)

In [10]:
#define the optimizer, metric we try to optimize, and how we calculate loss
reg = tflearn.regression(linear, optimizer='sgd', loss='mean_square', metric='R2', learning_rate=0.01, name="regression")# set the learning rate, default is off

In [11]:
model = tflearn.DNN(reg)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [12]:
#train the model with training data
model.fit(train_house_size_norm, train_price_norm, n_epoch=1000)

Training Step: 1999  | total loss: 0.04127 | time: 0.001s
| SGD | epoch: 1000 | loss: 0.04127 -- iter: 064/112
Training Step: 2000  | total loss: 0.04109 | time: 0.003s
| SGD | epoch: 1000 | loss: 0.04109 -- iter: 112/112
--


In [13]:
#Output W and b for the trained linear equation
print(" Weights: W={0}, b={1}\n".format(model.get_weights(linear.W), model.get_weights(linear.b)))

 Weights: W=[0.9787439], b=[-0.00142506]



In [14]:
#Evaluate Accuracy
print("Accuracy {0} ".format(model.evaluate(test_house_size_norm,test_house_price_norm)))

Accuracy [0.9579417109489441] 


In [15]:
model.save('tflearn_regression.tflearn')

INFO:tensorflow:/Users/jangalask/Desktop/Interpretability/tensorflow_learnings/tflearn_regression.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [17]:
model.load('./tflearn_models/tflearn_regression.tflearn')

INFO:tensorflow:Restoring parameters from /Users/jangalask/Desktop/Interpretability/tensorflow_learnings/tflearn_models/tflearn_regression.tflearn


In [20]:
model.predict(test_house_size_norm)

array([-0.7522289 , -1.6219    , -1.9603076 , -0.4465705 ,  1.0871799 ,
        0.3648799 , -0.76496464, -1.3107833 , -0.4793196 , -2.1622605 ,
        1.1635946 ,  0.8615749 ,  1.80948   ,  0.7178427 , -1.3526294 ,
       -0.4083632 , -0.18275815,  0.7978961 ,  0.97437745,  0.57047164,
       -0.88868356, -0.42291835,  0.65416384,  0.14473306,  1.2691195 ,
        0.1283585 ,  0.6905517 ,  0.4467527 ,  1.372825  ,  0.38671264,
        1.7348847 ,  0.22660585,  0.21205068,  0.0410275 , -0.56665057,
        0.8815883 , -0.00263799, -1.0451516 ,  0.35578293,  0.27391014,
       -1.1561347 , -1.7583547 , -0.04812288, -1.0924559 , -1.1288438 ,
        1.3000493 ,  0.6323311 ,  0.38489327], dtype=float32)

In [21]:
test_house_size_norm

array([-7.67109625e-01, -1.65566795e+00, -2.00142495e+00, -4.54812977e-01,
        1.11224699e+00,  3.74260270e-01, -7.80121986e-01, -1.33779457e+00,
       -4.88273332e-01, -2.20776381e+00,  1.19032116e+00,  8.81742324e-01,
        1.85023372e+00,  7.34888542e-01, -1.38054947e+00, -4.15775895e-01,
       -1.85271226e-01,  8.16680522e-01,  9.96994658e-01,  5.84316944e-01,
       -9.06527772e-01, -4.30647164e-01,  6.69826741e-01,  1.49332326e-01,
        1.29813786e+00,  1.32602148e-01,  7.07004913e-01,  4.57911158e-01,
        1.40409565e+00,  3.96567173e-01,  1.77401846e+00,  2.32983214e-01,
        2.18111945e-01,  4.33745345e-02, -5.77500946e-01,  9.02190319e-01,
       -1.23927242e-03, -1.06639391e+00,  3.64965726e-01,  2.81314838e-01,
       -1.17978734e+00, -1.79508609e+00, -4.77119880e-02, -1.11472554e+00,
       -1.15190371e+00,  1.32973930e+00,  6.47519837e-01,  3.94708264e-01])